In [1]:
from hpelm import ELM
import numpy as np
import time
from sklearn.utils import shuffle

In [2]:
# from PIL import Image
# from numpy import array
# from scipy.misc import imresize

In [3]:
# from os import listdir
# from os.path import isfile, join
# from cutImages import cut_image


# def read_images(path):
#     fullList = []
#     for c in range(1, 100):
#         mypath = path + str(c) + "/"
#         onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#         print (len(onlyfiles), c)
#         for x in onlyfiles:
#             if x == 'bb_info.txt':
#                 continue
#             img = cut_image(mypath + x)
#             arr = array(img)
#             arr = imresize(arr, [32, 32, 3])
#             r = arr[:, :, 0].flatten()
#             g = arr[:, :, 1].flatten()
#             b = arr[:, :, 2].flatten()
#             label = [c]
#             out = np.array(list(label) + list(r) + list(g) + list(b), np.int)
#             fullList.append(out)
#     shuffle(fullList)
#     imagesList = np.array(fullList, dtype=int)
#     print ("images shape: {}".format(imagesList.shape))
#     return imagesList

In [2]:
# imagesList = np.load('UECFOOD_100_1000Features_train.npy')
trainingdata = np.loadtxt('/media/aymen/DATA/datasets/food10/2048trainingdata-cnn.svm')

In [3]:
trainingdata.shape

(9000, 2049)

In [2]:
from cPickle import load
all_features_file = '/media/aymen/DATA/datasets/ghalib-features/data_features_orb_UECFOOD100.pickle'
all_features = load(open(all_features_file, 'rb'))

In [6]:
img_class = all_features[0][0].split('/')[-2]
print(img_class)

NameError: name 'all_features' is not defined

In [32]:
fet = all_features[0][1]
print fet.shape

(500, 32)


In [4]:
count = 0 
for i, val in all_features:
    if count < 3 :
        img_class = i.split('/')[-2]
        print img_class, val
        count +=1
    else:
        break
        

NameError: name 'all_features' is not defined

In [103]:
imagesList = shuffle(trainingdata)
precentage = 0.1
print ('imaglist full', imagesList.shape)
trainImagesList = imagesList[0:int(imagesList.__len__() * precentage), :]
testImagesList = imagesList[int(imagesList.__len__() * precentage):, :]
print ('trainImagesList', trainImagesList.shape)
print ('testImagesList', testImagesList.shape)
print (trainImagesList.shape[0] + testImagesList.shape[0])
print(imagesList[324])
print(np.unique(imagesList[:,0]))

('imaglist full', (9000, 2049))
('trainImagesList', (900, 2049))
('testImagesList', (8100, 2049))
9000
[ 5.        0.065093  0.014931 ...,  0.007085  0.236203  0.048631]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


In [104]:
# images_norm = imagesList[:,1:]
# labels = imagesList[:,0]
# from sklearn.preprocessing import normalize
# images_norm = normalize(images_norm)

# print (images_norm.shape)
# print('labels',labels.shape)
# print(images_norm[324])
# print(np.unique(images_norm))
# trainImagesList = images_norm[0:int(images_norm.__len__() * .8), :]
# testImagesList = images_norm[int(images_norm.__len__() * .8):, :]

# trainImagesList_labels = labels[0:int(labels.__len__() * .8)]
# testImagesList_labels = labels[int(labels.__len__() * .8):]

# print ('trainImagesList', trainImagesList.shape)
# print ('testImagesList', testImagesList.shape)
# print ('trainImagesList_labels', trainImagesList_labels.shape)
# print ('testImagesList_labels', testImagesList_labels.shape)
# # images = norm2


In [105]:
epoch = 1
batch_size = trainImagesList.__len__() / epoch
hidden_num = 5000
_inputs = 2048
_outputs = 10
elm = ELM(_inputs, _outputs, batch=batch_size)
elm.add_neurons(hidden_num, "tanh")
print("batch_size : {}".format(batch_size))
print("hidden_num : {}".format(hidden_num))

batch_size : 900
hidden_num : 5000


In [106]:
k=1
old_batch_size_k = 0
data = trainImagesList
start_time = time.time()
while k <= epoch:
    # shuffle(trainImagesList)
    print("batch : {}".format(k))
    # print (data.shape, data.dtype)
    # dynamic
#     train_x = np.array(data[old_batch_size_k:(batch_size * k), 1:], dtype="float")
#     train_y = np.array(data[old_batch_size_k:(batch_size * k), 0], dtype="int")
#     if k == epoch:
#         train_x = np.array(data[old_batch_size_k:, 1:], dtype="float")
#         train_y = np.array(data[old_batch_size_k:, 0], dtype="int")
#     old_batch_size_k = batch_size * k

    # whole
    train_x = np.array(data[:, 1:], dtype="float")
    train_y = np.array(data[:, 0], dtype="int")
    #separate labels and images 
#     train_x = np.array(trainImagesList, dtype="float")
#     train_y = np.array(trainImagesList_labels, dtype="int")
    
    print ("X", train_x.shape)
    print ("Y", train_y.shape)
    # end whole
    train_y = np.eye(np.max(train_y) + 1)[train_y]
    elm.train(train_x, train_y)
    k += 1

end_time = time.time()
print("%s seconds" % (end_time - start_time))

batch : 1
('X', (900, 2048))
('Y', (900,))
Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons
86.8410181999 seconds


In [107]:
def predict(_testImagesList):
    # testImagesList = np.load(testPath)
    testList = _testImagesList

    # print ('testImagesList', testList, testList.shape)
    test_x = np.array(testList[:, 1:], dtype="float")
    test_y = np.array(testList[:, 0], dtype="int")
    test_y = np.eye(np.max(test_y) + 1)[test_y]
    #
    Y = elm.predict(test_x)
    predict = []
    for y in Y:
        predict.append(y.argmax())

    gt = []
    for y in test_y:
        gt.append(y.argmax())

    save = []
    for _ in np.arange(0, len(gt)):
        k = True if gt[_] == predict[_] else False
        save.append(k)

    return save

In [108]:
# def predict_labels(_testImagesList,_labels):
#     # testImagesList = np.load(testPath)
#     testList = _testImagesList
#     labels = _labels

#     # print ('testImagesList', testList, testList.shape)
#     test_x = np.array(testList, dtype="float")
#     test_y = np.array(labels, dtype="int")
#     test_y = np.eye(np.max(test_y) + 1)[test_y]
#     #
#     Y = elm.predict(test_x)
#     predict = []
#     for y in Y:
#         predict.append(y.argmax())

#     gt = []
#     for y in test_y:
#         gt.append(y.argmax())

#     save = []
#     for _ in np.arange(0, len(gt)):
#         k = True if gt[_] == predict[_] else False
#         save.append(k)

#     return save

In [109]:
test = predict(testImagesList)
print("test accuracy: {0}".format(np.mean(test)))
train = predict(trainImagesList)
print("train accuracy: {0}".format(np.mean(train)))

test accuracy: 0.833827160494
train accuracy: 1.0


In [46]:
# test = predict_labels(testImagesList,testImagesList_labels)
# print("test accuracy: {0}".format(np.mean(test)))
# train = predict_labels(trainImagesList,trainImagesList_labels)
# print("train accuracy: {0}".format(np.mean(train)))

In [ ]:
test accuracy: 0.867333333333
train accuracy: 0.992444444444